# First fabric experiment

Prerequisites: Create an account with the fabric testbed. Please follow all the instructions given in <code> configure_environment.ipynb</code> which can be easily located via <code> start_here.ipynb</code> inside the <code> fabric_example</code> folder. <code> configure_environment.ipynb</code> contains instructions about setting up the fabric environment.

### Import the fabric library

The fabric library contains functions that can be used to access all the underlying API. 

In [3]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib= fablib_manager()

# This function is not necessary but can be used to see the environment settings
fablib.show_config()

-----------------------------------  --------------------------------------------------
credmgr_host                         cm.fabric-testbed.net
orchestrator_host                    orchestrator.fabric-testbed.net
fabric_token                         /home/fabric/.tokens.json
project_id                           68926660-da26-475d-9c40-50ebf0a5a812
bastion_username                     mjdbz4_0000018266
bastion_key_filename                 /home/fabric/work/fabric_config/fabric_bastion_key
bastion_public_addr                  bastion-1.fabric-testbed.net
bastion_passphrase                   None
slice_public_key_file                /home/fabric/work/fabric_config/slice_key.pub
slice_private_key_file               /home/fabric/work/fabric_config/slice_key
fabric_slice_private_key_passphrase  None
fablib_log_file                      /tmp/fablib/fablib.log
fablib_log_level                     INFO
-----------------------------------  --------------------------------------------------


### Finding Resources:

To find available resources in the fabric testbed.

In [5]:
try:
    print(f"{fablib.list_sites()}")
except Exception as e:
    print(f"Exception: {e}")

Name      CPUs  Cores    RAM (G)    Disk (G)       Basic (100 Gbps NIC)    ConnectX-6 (100 Gbps x2 NIC)    ConnectX-5 (25 Gbps x2 NIC)    P4510 (NVMe 1TB)    Tesla T4 (GPU)    RTX6000 (GPU)
------  ------  -------  ---------  -------------  ----------------------  ------------------------------  -----------------------------  ------------------  ----------------  ---------------
GPN         10  300/320  2520/2560  116300/116400  635/635                 2/2                             0/0                            16/16               4/4               6/6
TACC        10  258/320  2332/2560  115420/116400  597/635                 2/2                             4/4                            16/16               4/4               5/6
NCSA         6  174/192  1464/1536  60490/60600    378/381                 2/2                             2/2                            10/10               2/2               3/3
STAR        12  352/384  2968/3072  121050/121200  755/762                 2/2

### Creating a slice

A slice is a collection of compute resources stitched together to form some topology. In this experiment we will be creating a single node, with basic compute resources added to it. We can choose the resources from the table shown above (output of Finding Resources section). Experimenters usually get equal or more than the requested amount of compute resources.

In [14]:
# Naming the resources
slice_name="First_slice"
node1_name="Node1"

In [15]:
try:
    slice=fablib.new_slice(name=slice_name)
    
    node1=slice.add_node(name=node1_name,
                         site='STAR', # name of the site, please see above table for other options.
                         host='star-w2.fabric-testbed.net', # address of the STAR site worker
                         cores=4, # number of cores
                         ram=8, # amount of RAM
                         disk=10, # Disk space
                         image='default_ubuntu_20') # operation system type
    slice.submit()
except Exception as e:
    print(f"Exception: {e}")


-----------  ------------------------------------
Slice Name   First_slice
Slice ID     5a924cc7-96b4-46a9-baac-d998cc604644
Slice State  StableOK
Lease End    2022-10-11 15:34:54 +0000
-----------  ------------------------------------

Retry: 12, Time: 133 sec

ID                                    Name    Site    Host                          Cores    RAM    Disk  Image              Management IP                           State    Error
------------------------------------  ------  ------  --------------------------  -------  -----  ------  -----------------  --------------------------------------  -------  -------
14f1c3a3-441e-45c3-abe2-be3c65d4387e  Node1   STAR    star-w2.fabric-testbed.net        4      8      10  default_ubuntu_20  2001:400:a100:3030:f816:3eff:fe3f:f58d  Active

Time to stable 133 seconds
Running post_boot_config ... Time to post boot config 135 seconds


### List Slice Attributes

In [16]:
try:
    print(f"{slice}")
except Exception as e:
    print(f"Exception: {e}")

-----------  ------------------------------------
Slice Name   First_slice
Slice ID     5a924cc7-96b4-46a9-baac-d998cc604644
Slice State  StableOK
Lease End    2022-10-11 15:34:54 +0000
-----------  ------------------------------------


### List Node Attributes

In [17]:
try:
    for node in slice.get_nodes():
        print(f"{node}")
except Exception as e:
    print(f"Exception: {e}")

-----------------  ------------------------------------------------------------------------------------------------------------------------------------------------
ID                 14f1c3a3-441e-45c3-abe2-be3c65d4387e
Name               Node1
Cores              4
RAM                8
Disk               10
Image              default_ubuntu_20
Image Type         qcow2
Host               star-w2.fabric-testbed.net
Site               STAR
Management IP      2001:400:a100:3030:f816:3eff:fe3f:f58d
Reservation State  Active
Error Message
SSH Command        ssh -i /home/fabric/work/fabric_config/slice_key -J mjdbz4_0000018266@bastion-1.fabric-testbed.net ubuntu@2001:400:a100:3030:f816:3eff:fe3f:f58d
-----------------  ------------------------------------------------------------------------------------------------------------------------------------------------


### Running a simple experiment

A simple experiment to add two numbers.

In [18]:
try:
    node=slice.get_node(name=node1_name)
    stdout, stderr=node.execute("echo '1 + 1' | bc")
    
    print(stdout)
    print(stderr)
    
except Exception as e:
    print(f"Excetion: {e}")

2




### Delete the slice

Delete the slice after the experiment is over.

In [13]:
try:
    slice=fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Exception: {e}")